# Составление словарей для классификации по тональности

## Импорт исходных данных, составление dataframe

In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:

responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [02:43, 1226.95it/s]


In [4]:
counter = {}
for resp in responses:
    if resp['bank_name'] in counter:
        counter[resp['bank_name']] += 1
    else:
        counter[resp['bank_name']] = 1

In [5]:
df = pd.DataFrame()

lens = []
symbols = []
texts = []
for resp in responses:
    symbols.append(len(resp['text']))
    lens.append(len(resp['text'].split(' ')))
    texts.append(resp['text'])

grades = []
for resp in responses:
    grades.append(resp['rating_grade'])

df['lens'] = lens
df['symbols'] = symbols
df['texts'] = texts
df['grades'] = grades

In [6]:
from nltk.tokenize import sent_tokenize

### Так как данных очень много, а у меня слабый ноут и виндовс, оставим только часть данных (с метками 1 и 5)

In [7]:
df1 = df[(df.grades == 1) | (df.grades ==5)]

In [8]:
df1.shape[0], df.shape[0]

(62100, 153499)

In [9]:
df1['sents'] = df1.texts.apply(sent_tokenize)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
sentences = []
for _, row in df1.iterrows():
    for sent in row.sents:
        sentences.append(sent)
    

In [11]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""


# df.texts = df.texts.str.lower()
# df.texts = df.texts.apply(words_only)

### Выделим предложения и лемматизируем их. Лемматизация опять сломалась, поэтому закомментирована

In [12]:
for i in range(len(sentences)):
    sentences[i] = sentences[i].lower()
    sentences[i] = words_only(sentences[i])

In [13]:
sentences[100]

'разумеется один из клиентов задал вопрос сотрудникам банка а мол почему все так долго'

In [14]:
from pymorphy2 import MorphAnalyzer

In [15]:
from nltk.corpus import stopwords

mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

def lemmatize(text):
    m = MorphAnalyzer()
    try:
        lemmas1 = [m.parse(word)[0].normal_form for word in text.split()]
        return ' '.join(lemmas1) 
    except:
        return " "

mystoplemmas = ['который','прошлый','сей', 'свой', 'наш', 'мочь']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

In [16]:
# for i in tqdm(range(len(sentences))):
#     sentences[i] = remove_stopwords(sentences[i])
#     sentences[i] = lemmatize(sentences[i])

In [17]:
texts = [sentences[i].split() for i in range(len(sentences))]

## Обучение word2vec

In [18]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)
model.save("sent_w2v.model")

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 2min 4s


### Примеры тестов

In [19]:
model.most_similar("магазин")[:4]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('салон', 0.8329899907112122),
 ('автосалон', 0.7703304290771484),
 ('допофис', 0.6486024856567383),
 ('сбер', 0.645979106426239)]

In [20]:
model.most_similar(positive=["стул","надежный"], negative=["плохой"])[:4]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('диван', 0.6388599276542664),
 ('стульчик', 0.5988638401031494),
 ('путь', 0.5562049746513367),
 ('оперзал', 0.5489509105682373)]

In [21]:
model.most_similar(positive=["сотрудник","вежливый"], negative=["хам"])[:4]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('представитель', 0.5858386754989624),
 ('специалист', 0.5764997005462646),
 ('консультант', 0.5584383606910706),
 ('работник', 0.5549967288970947)]

In [22]:
model.doesnt_match("борщ сметана макароны пирожок кровать".split())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'кровать'

### Визуализация

In [23]:
top_words = []
from nltk import FreqDist
fd = FreqDist()
for text in texts:
    fd.update(text)
for i in fd.most_common(1000):
    top_words.append(i[0])

In [24]:
top_words_vec = model[top_words]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [70]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

#### Кластеры слов

In [71]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

#### Названия банков (с некоторыми лишним словами) по осям "хорошо-плохо" и "быстро-медленно"

In [91]:
banks = []
x1_b = []
x2_b = []
names_b = []
for key in counter:
    name = key.lower().split()
    for i in range(len(name)):
        try:            
            x = (model['хорошо'] - model['плохо']).dot(model[name[i]])
            y = (model['быстро'] - model['медленно']).dot(model[name[i]])
            banks.append([name[i],x,y])
            x1_b.append(x)
            x2_b.append(y)
            names_b.append(name[i])
        except:
            continue

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [93]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=x1_b,
                                    x2=x2_b,
                                    names=names_b))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

C:\ProgramData\Anaconda3\lib\site-packages\bokeh\core\json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


## Распространение метки

### Составляем df из топ-1000 слов и размечаем в ней метку вручную.
Из-за того, что слов осталось мало, метку разметил по 2 хорошим и 2 плохим словам

In [38]:
big_data = []
for i in range(len(top_words)):
    row = []   
    for j in range(len(top_words_vec[i])):
        row.append(top_words_vec[i][j])
    row.append(top_words[i])
    big_data.append(row)

In [39]:
# big_data = []
# for key in top_words_vec.wv.vocab:
#     row = model.wv.get_vector(key).tolist()
#     row.append(key)
#     big_data.append(row)

In [105]:
df2 = pd.DataFrame.from_records(big_data)

In [106]:
# df2 = df2.set_index(100)

In [76]:
# df2.to_csv('w2v.csv')

In [107]:
# df2 = pd.read_csv('w2v.csv')

In [108]:
df2['label'] = -1

In [109]:
good_words = ['быстрый', 'качественно', 'надежно', 'профессионал', 'отзывчивый', 'быстро','хорошо']
bad_words = ['медленно', 'медленный', 'дорого', 'очередь', 'ненадежный', 'плохо','ужас','глупый','долго']

In [110]:
# df2[df2['100'].isin(good_words)] = 1

df2.loc[df2[100].isin(good_words),['label']] = 1
df2.loc[df2[100].isin(bad_words),['label']] = 0
df2[df2['label'] == 1]

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,label
228,1.329919,0.744598,-0.307002,3.819704,0.414438,-0.321052,-0.455334,-1.736185,3.879706,0.086049,...,-1.578944,3.689738,-1.367663,-1.365734,-2.305943,-1.394026,-1.507384,-0.508937,быстро,1
300,1.255101,0.177312,0.742934,0.955450,-1.202813,0.606358,1.408049,-2.313519,0.570257,0.964073,...,-1.520482,2.306047,-0.729101,0.683989,-0.993014,-1.494889,-0.411660,1.780157,хорошо,1


### Составление исходных данных для обучения и обучение алгоритма.

In [114]:
df2 = df2.set_index(100)

In [115]:
y = df2['label']

In [116]:
df2 = df2.drop(['label'], axis = 1)

In [163]:
from sklearn.semi_supervised import LabelSpreading

la = LabelSpreading(gamma=1)

In [164]:
# df1.to_csv('df1.csv')

In [165]:
# from sklearn.semi_supervised import LabelPropagation

# la = LabelPropagation(n_neighbors=2, tol=0.001,kernel = 'knn')

In [166]:
from scipy.sparse.csgraph import connected_components

In [167]:
la.fit(df2,y.to_dense())

LabelSpreading(alpha=0.2, gamma=1, kernel='rbf', max_iter=30, n_jobs=1,
        n_neighbors=7, tol=0.001)

In [168]:
df2['new_label'] = la.predict(df2)

### Итоговые предсказания

In [169]:
list(df2[df2['new_label']==1].index)

['в',
 'и',
 'не',
 'на',
 'что',
 'я',
 'с',
 'по',
 'мне',
 'банка',
 'а',
 'банк',
 'как',
 'за',
 'но',
 'меня',
 'это',
 'к',
 'все',
 'у',
 'о',
 'так',
 'деньги',
 'было',
 'же',
 'от',
 'для',
 'через',
 'уже',
 'из',
 'до',
 'карту',
 'карты',
 'т',
 'только',
 'при',
 'нет',
 'после',
 'кредит',
 'если',
 'отделение',
 'заявление',
 'банке',
 'время',
 'она',
 'г',
 'вы',
 'со',
 'счет',
 'этом',
 'они',
 'или',
 'будет',
 'без',
 'мы',
 'там',
 'их',
 'есть',
 'этого',
 'вот',
 'ответ',
 'почему',
 'вопрос',
 'карта',
 'том',
 'рублей',
 'мой',
 'он',
 'можно',
 'сумму',
 'его',
 'может',
 'сказали',
 'денег',
 'сегодня',
 'офис',
 'этот',
 'смс',
 'счета',
 'того',
 'вам',
 'банком',
 'ее',
 'могу',
 'сотрудники',
 'руб',
 'кредита',
 'да',
 'нужно',
 'надо',
 'ну',
 'чем',
 'который',
 'сотрудник',
 'более',
 'тут',
 'девушка',
 'об',
 'документы',
 'мои',
 'хотя',
 'е',
 'вклад',
 'карте',
 'сбербанка',
 'клиентов',
 'сотрудников',
 'телефону',
 'один',
 'сразу',
 'опять'

Резюме: работа выполнена с рядом неточностей, связанными с трудностями реализации на windows и временем выполнения. Однако, ясно что нужно сделать при наличии большего времени и лучшей настройки системы:
1. Выполнить правильное извлечение токенов - удалить стоп-слова и провести лемматизацию
2. Обучать w2v на всем объеме данных
3. Разобаться с построением графа распространения метки, учесть вероятность распространения, чтобы получать метку не для всех слов, а только для части.